# Bart WalkThrough
> A journey through the forward pass

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: images/chart-preview.png

### Seq2Seq Pretraining
In October 2019, teams from Microsoft, Google and Facebook independently published three new transformer papers: [UniLM](https://arxiv.org/abs/1905.03197), [T5](https://arxiv.org/abs/1910.10683) and [Bart](https://arxiv.org/abs/1910.13461). All three papers found that they achieve better downstream performance on generation tasks if they (a) replace Bert's bidirectional architecture with a seq2seq architecture and (b) Bert's fill-in-the blank cloze task with a more complicated mix of pretraining tasks. 

Now lets dig deeper into the big Seq2Seq pretraining idea, then dive into some interesting parts of the `BartForConditionalGeneration` code.

### Background: Bert vs. GPT2
As the Bart authors' write, 
> (Bart) can be seen as generalizing Bert (due to the bidirectional encoder) and GPT (with the left to right decoder). 


Bert is pretrained to try to predict masked tokens, and uses the whole sequence to get enough info to make a good guess. This is good for tasks where the prediction at position `i` is allowed to utilize information from positions after `i`, but less useful for tasks where you are not, like text generation, where you generate the next word conditional on the previously generated words.

In code, the idea of "what information you can use when predicting the token at position `i`" is controlled by a parameter called `attention_mask`[^2]. 
> Note: In this post, we show attention masks in grids where each row `y` represents an output token, and each column `x` represents an input token. If the square at `(y3, x4)` is black. It means that our prediction for `y3` is allowed to utilize information from `x4`. During pretraining, `x` would be the corrupted document, and `y` would be the original.


Bert's "Fully-visible" `attention_mask` is boring:
![](./bert_mac_small.jpg)


[^2]: the same parameter that is used to make model predictions invariant to pad tokens.

GPT2, meanwhile, is pretrained to predict the next word.
This makes it good for producing generations, where there aren't future tokens to consider, but less useful for other downstream tasks where the whole sentence is utilized.

Here is the `attention_mask` for GPT2:

![](./gpt2_simple_mask.jpg)

<!-- All together, now:
![](./t5_mask_diagram.png) -->


### Encoder-Decoder
Our new friends get the best of both worlds!

The encoder is bidirectional  - each token's attention can attend to every other token in the input sequence, while the decoder, which will ultimately have to perform generation, is causal like GPT2.

![](./causal_with_prefix.jpg)


We can think about this `attention_mask` as smushing together our previous two attention masks, or "Causal Mask  with a fully visible prefix" in fancier terms.[^4]

[^4]: The indices dont line up perfectly for the smush to work, but tokens 1 and 2 are the fully visible prefix (or the input to the encoder) and tokens 3,4,5 are the causally masked suffix (or inputs to the decoder). In summarization terms, you could imagine tokens 1 and 2 as the article, and we generate tokens 3-5 auto-regressively.

Now that we understand the big idea, we are going to focus on using Bart for Summarization.


![](./t5_mask_diagram.png)


### Summarization

During summarization finetuning, the `source` sequence is the document we want to summarize, and the `target` sequence is a ground truth summary.
The Seq2Seq attention pattern is therefore very well suited to summarization and other conditional generation tasks. We are allowed to attend to the whole source document, but as we write your summary, one word at a time, we can only consider what we've already written. The numbers confirm this: all the new fancy guys do a lot better than the old less-fancy guys on the CNN/Daily Mail summarization task.

In [78]:
#collapse-hide 
import pandas as pd
#df = pd.read_csv('tab1.csv', index_col=0)
df = pd.DataFrame(dict(rouge2 = {'Bart': 21.28,
 'UniLM': 20.3,
 'BertSumABS': 19.39,
 'T5-base': 20.34,
 'T5-11B': 21.55,
 'TransformerAbs': 17.76},
model_size = {'Bart': "406 M",
 'UniLM': "340 M",
 'BertSumABS': "220 M",
 'T5-base': "770 M",
 'T5-11B': "11 B",
 'TransformerAbs': "200M"}
    )).sort_values('rouge2')

df['Pretraining'] = ['None', 'Encoder'] + ['Seq2Seq']*4
df.rename_axis('Model').rename(columns={'model_size': 'Model Size', 'rouge2': 'Rouge-2'})

,Rouge-2,Model Size,Pretraining
Model,,,
TransformerAbs,17.76,200M,None
BertSumABS,19.39,220 M,Encoder
UniLM,20.30,340 M,Seq2Seq
T5-base,20.34,770 M,Seq2Seq
Bart,21.28,406 M,Seq2Seq
T5-11B,21.55,11 B,Seq2Seq


<!-- I start with this not to say that the researchers are lame, but that they were all discovering a very cool idea at the same time. In the following table, you can see that, at least for summarization, it is important to pretrain with a bidirectional encoder and causal decoder: (The UniLM `prefix_lm` is equivalent [^3]) -->


> Note: `BertSumABS` (from [*Text Summarization with Pretrained Encoders*](https://arxiv.org/abs/1908.08345), uses a Seq2Seq architecture but doesn't pretrain the decoder.

Shortly after these papers were released, people wanted to use them in `transformers`, with lots of attention dedicated to Bart, which we introduce today.

Bart tries out crazy pretraining tasks that you can only do with a seq2seq architecture. Since "Inputs to the encoder need not be aligned with decoder outputs, allowing arbitary noise transformations." They use a pretraining task called Text Infilling, where you
replace a **span** of text with a single mask token. This span can be of any length, so the model also must learn how many tokens to generate.

There is also another trick in Bart: each decoder layer performs cross over the final hidden state of the encoder output. This presumably nudges Bart towards generating summaries that are closely connected to the original (encoded) text.

And now, lets use `BartForConditionalGeneration` to summarize a boring article about tennis from the CNN dataset.

### `BartForConditionalGeneration`

**imports**

In [85]:
#collapse-hide
import torch
import transformers  #  Installed from source at commit e03129ad
from transformers import BartTokenizer, BartForConditionalGeneration
from IPython.display import display, Markdown

torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
def print_80_per_line(txt, n=80):
    result = []
    for i in range(0, len(txt), n):
        result.append(txt[i:i + n])
    return '\n'.join(result)




transformers installed from source
-e git+git@github.com:sshleifer/transformers_fork.git@4f9d1f20c3950d81e097c1409d8165a2577fcf32#egg=transformers


In [41]:
#collapse-show
LONG_BORING_TENNIS_ARTICLE = """
 Andy Murray  came close to giving himself some extra preparation time for his w
edding next week before ensuring that he still has unfinished tennis business to
 attend to. The world No 4 is into the semi-finals of the Miami Open, but not be
fore getting a scare from 21 year-old Austrian Dominic Thiem, who pushed him to 
4-4 in the second set before going down 3-6 6-4, 6-1 in an hour and three quarte
rs. Murray was awaiting the winner from the last eight match between Tomas Berdy
ch and Argentina's Juan Monaco. Prior to this tournament Thiem lost in the secon
d round of a Challenger event to soon-to-be new Brit Aljaz Bedene. Andy Murray p
umps his first after defeating Dominic Thiem to reach the Miami Open semi finals
 . Muray throws his sweatband into the crowd after completing a 3-6, 6-4, 6-1 vi
ctory in Florida . Murray shakes hands with Thiem who he described as a 'strong 
guy' after the game . And Murray has a fairly simple message for any of his fell
ow British tennis players who might be agitated about his imminent arrival into 
the home ranks: don't complain. Instead the British No 1 believes his colleagues
 should use the assimilation of the world number 83, originally from Slovenia, a
s motivation to better themselves. At present any grumbles are happening in priv
ate, and Bedene's present ineligibility for the Davis Cup team has made it less 
of an issue, although that could change if his appeal to play is allowed by the 
International Tennis Federation. Murray thinks anyone questioning the move, now 
it has become official, would be better working on getting their ranking closer 
to his. 'If he was 500 in the world they wouldn't be that fussed about it but ob
viously he threatens their position a bit,' said the 27 year-old Scot. ' and he'
s obviously the British number two, comfortably. 'So they can complain but the b
est thing to do is use it in the right way and accept it for what it is, and try
 to use it as motivation whether they agree with it or not. He's British now so 
they've just got to deal with it. Murray stretches for a return after starting h
is quarter final match slowly on the show court . Thiem held nothing back as he 
raced through the opening set, winning it 6-3 with a single break . The young Au
strian is considered to be one of the hottest prospects on the ATP Tour . 'I wou
ld hope that all the guys who are below him now like James (Ward) , Kyle (Edmund
) , Liam (Broady) they will use it as motivation. If he becomes eligible for Dav
is Cup then those guys are going to have to prove themselves. 'It can only be se
en as a positive for those guys using it to try to get better. He's a good playe
r but so are James and Kyle and Liam has improved. Aljaz is there, he's on the t
our every week, the other guys aren't quite there yet.' For the first time Murra
y, who has an encyclopaedic knowledge of the top 100, gave his opinion of Bedene
: 'He's a good player with a very good serve. He's a legitimate top 100 player, 
when he plays Challengers he's there or thereabouts, when he plays on the main t
our he wins matches, it's not like he turns up and always loses in the first rou
nd. Murray's fiancee was once again watching from the stands shaded by a huge br
immed hat . Kim Sears flashes her enormous diamond engagement ring while watchin
g her beau on court . 'He had a bad injury last year (wrist) but has recovered w
ell. I would imagine he would keep moving up the rankings although I don't know 
exactly how high he can go. I've practised with him a couple of times, I haven't
 seen him play loads, but when you serve as well as he does it helps. I would im
agine he' s going to be comfortably in the top 70 or 80 in the world for a while
.' It is understood the Lawn Tennis Association will give background support to 
his case regarding the Davis Cup but have made it clear that the onus is on him 
to lead the way. An official statement said: 'To have another player in the men'
s top 100 is clearly a positive thing for British tennis and so we very much wel
come Aljaz's change in citizenship.' The last comparable switch came twenty year
s ago when Greg Rusedski arrived from Canada. It was by no means universally pop
ular but, like Bedene, he pledged that he was in for the long haul and, in fairn
ess to him, he proved true to his word. Loising the first set shocked Murray int
o life as he raced to a commanding lead in the second . The No 3 seed sent over 
a few glaring looks towards his team before winning the second set . Murray had 
to put such matters aside as he tackled the unusually talented Thiem, a delight 
to watch. Coached by Boris Becker's veteran mentor Gunter Bresnik, he slightly r
esembles Andy Roddick and hits with similar power but more elegance. His single 
handed backhand is a thing of rare beauty. However, he has had a mediocre season
 coming into this event and there was little to forewarn of his glorious shotmak
ing that seemed to catch Murray unawares early on. The world No 4 looked to have
 worked him out in the second, but then suffered one of his periopdic mental lap
ses and let him back in from 4-1 before closing it out with a break. After break
ing him for 3-1 in the decider the Austrian whirlwind burnt itself out. 'He's a 
strong guy who hits the ball hard and it became a very physical match,' said Mur
ray. Murray was presented with a celebratory cake after winning his 500th match 
in the previous round .
""".replace('\n','')

In [ ]:
tokenizer = BartTokenizer.from_pretrained('bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('bart-large-cnn')

In [98]:
article_input_ids = tokenizer.batch_encode_plus([LONG_BORING_TENNIS_ARTICLE], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
summary_ids = model.generate(article_input_ids, num_beams=4, length_penalty=2.0,
                             max_length=140, min_len=55)

In [77]:
summary_txt = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
display(Markdown('> **Summary:'+summary_txt))

> **Summary:** Andy Murray beat Dominic Thiem 3-6, 6-4, 6-1 in the Miami Open. The world No 4 is into the semi-finals of the tournament in Florida. Murray was awaiting the winner from the last eight match between Tomas Berdych and Argentina's Juan Monaco. Thiem lost in the second round of a Challenger event to Aljaz Bedene.

In [99]:
#collapse-show
from transformers import GPT2LMHeadModel, GPT2Tokenizer
gpt2_tok = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2', output_past=True)
enc = gpt2_tok.encode(LONG_BORING_TENNIS_ARTICLE, max_length=1024-155, return_tensors='pt')
summary_ids = gpt2_model.generate(enc, max_length=1024, do_sample=False)
source, generated = gpt2_tok.decode(summary_ids[0]).split("An official statement said:")

GPT2 (which in fairness is not finetuned for summarization) cannot really continue the tennis article sensically or generate process as many tokens.

In [130]:
display(Markdown('> **GPT2:** ' + generated))

> **GPT2:**  'To have a player like James Ward, Kyle Edmund, Liam Broady and Aljaz Bedene in the top 100 is a huge achievement for the Lawn Tennis Association. The Lawn Tennis Association is committed to the development of the sport and the development of the sport's players. The Lawn Tennis Association is committed to the development of the sport and the development of the sport's players. The Lawn Tennis Association is committed to the development of the sport and the development of the sport's players. The Lawn Tennis Association is committed to the development of the sport and the development of the sport's players. The Lawn Tennis Association is committed to the development of the sport and the development of the sport's players. The Lawn Tennis Association is committed to the development of the sport and the development of the

More importantly, these snippets show that even though `BartForConditionalGeneration` is a seq2seq model, and `GPT2LMHeadModel` is not, they can invoked in similar ways for generation.

> Note: The same correspondence exists between `BartForSequenceClassification`  and all the other `*ForSequenceClassification` in `transformers` -- they only require one argument: `input_ids`.

The full signature, however, is a little more complex.
```python
def forward(
    self,
    input_ids,
    attention_mask=None, # ignored pad tokens in input_ids
    decoder_input_ids=None, # make these if not supplied
    decoder_padding_mask=None, # ignored pad tokens in decoder_input_ids
    decoder_causal_mask=None, # Ignore future tokens in decoder_input_ids
):
```
During summarization finetuning, or seq2seq pretraining, we need to pass `decoder_input_ids`. (the masks will be made for you if you dont supply them).

When we're not, like in a classification context, you can safely ignore all the decoder kwargs and `BartModel` will make them for you[^5] 

[^5]: How? by taking the input_ids (movie review) and shifting them to the right. Random, I know. The authors' [motivation](https://github.com/pytorch/fairseq/issues/1389) for the shift-right trick was to facilitate teacher forcing during pre-training, and now that the model has been trained on 64 TPUs for 12 weeks to process this input format, we continue the pattern during inference, but hide it inside the forward method.

Now that we have an understanding of Bart, lets scurry over some interestring tricks we use to make generation faster.
> Warning: The following has a heavier engineering focus than the rest of the post

## Incremental Decoding

Anyways, here is a really slow (pseudocode) way to greedily generate summaries
    
```python
output_tokens = [bos]
while not done:
     encoder_hidden_state = model.encoder(article_input_ids)
     logits = model.decoder(encoder_hidden_state, output_tokens)
     next_word = logits.argmax()
     output_tokens.append(next_word)
     if next_word == eos: break
```

Let's say `N = len(article_input_ids)` and `M = len(output_tokens)` when we return. Let's also ignore the cost of encoder-decoder attention.
Then the complexity of this approach is roughly
- $(N \cdot M)$ (we call the encoder `M` times on `N` tokens)
-  $\sum_{m=1}^{M} m = \frac{M(M+1)}{2}$   (we call the encoder `M` times on 1 token then 2 tokens, etc).
- Total: $N \cdot M  + \frac{M(M+1)}{2}$



Let's say we are generating a 100 token summary of a 1024 token article, then we have to "process"
$N \cdot M  + \frac{M(M+1)}{2} =  107,450$ tokens

In [145]:
#collapse-hide
# show work
N,M=1024, 100
complexity_simple = int(N*M + ((M*(M+1))/2))
msg = "To generate a 100 token summary of a 1024 token article using this approach, we have to process {:,} tokens."
#display(Markdown(msg.format(complexity_simple)))

Thankfully, we can just hoist the `encoder` call outside the loop!

```python
output_tokens = []
encoder_hidden_state = model.encoder(article_input_ids)
while not done:
     logits = model.decoder(encoder_hidden_state, output_tokens)
     next_word = logits.argmax()
     
     output_tokens.append(next_word)
     if next_word == eos: break
```


Now the complexity is drastically reduced: $N \cdot 1  + \frac{M(M+1)}{2} = 6,074$

But we can actually go even further, and partially cache the attention outputs for the decoder.
This will change our generation loop to:
```python
output_tokens = []
encoder_hidden_state = model.encoder(article_input_ids)
cache = None
while not done:
     logits, cache = model.decoder(encoder_hidden_state, output_tokens[-1], cache=cache)
     next_word = logits.argmax()
     output_tokens.append(next_word)
     if next_word == eos: break
```
And change our formula to $N + M = 1,124$

The nitty-gritty of this trick is explained in the Bonus section below.

### Conclusion
Our first release of `BartModel` prioritized moving quickly and keeping the code simple. We want to see how the community uses it! As a result, our implementation is about 30% slower and uses more memory than the authors'. Stay tuned for episode 2 of this series, where we try to close the gap. 

A big thank you to Sasha Rush and Patrick Von Platten for reading many versions of this post, and to the Bart authors for providing their code on [fairseq](https://github.com/pytorch/fairseq/tree/master/examples/bart) and answering questions on github.


### Bonus: Partially caching k and v in `DecoderLayer`



Here is some pseudocode for attention without all the reshapes and heads and masks and scaling.


```python
class SimplifiedAttention(nn.Module):
    def __init__(self, embed_dim):
        self.Wq = torch.nn.Linear(embed_dim, embed_dim)
        self.Wk = torch.nn.Linear(embed_dim, embed_dim)
        self.Wv = torch.nn.Linear(embed_dim, embed_dim)
        self.dense = torch.nn.Linear(embed_dim, embed_dim)
    def forward(self, query, key, value):
        q = self.Wq(q)
        k = self.Wk(k) 
        v = self.Wv(v)
        matmul_qk = torch.matmul(q, k.T)
        attention_weights = matmul_qk.softmax(dim=-1)
        output = torch.matmul(attention_weights, v)
        return self.dense(output)
```

Now lets glimpse at the callers inside bart's `DecoderLayer`:
(LayerNorms and dropouts deleted for simplicity). Here's some more pseudocode

```python
class SimplifiedDecoderLayer(nn.Module):
    
    def __init__(self, embed_dim):
        self.self_attn = SimplifiedAttention(embed_dim)
        self.encoder_attn = SimplifiedAttention(embed_dim)
    def forward(x, last_encoder_hidden_state, *masks_etc):
         # x shape `(batch_size, tokens_generated_so_far, embed_dim)`
         # x comes from decoder

        x = self.self_attn(query=x, key=x, value=x) # pay attention to somebody else for a change!
        output = self.encoder_attn(
            query=x,
            key=last_encoder_hidden_state,  # could be None
            value=last_encoder_hidden_state,
        )
        return output
```

What did we learn?


- In `encoder_attention`, we can cache everything that doesn't depend on q, namely these outputs
```
        k = self.Wk(k) 
        v = self.Wv(v)
```


The more exciting optimization is that in `self_attn`, we can cache the part of k,v that depends on 
`x[:, :1]` the tokens we've already generated. Then each time through the generation loop, we only pass in `x[:, :-1]` and apply concatenation:

```python
k = torch.cat((past_key, new_k), dim='seq_len') # the seq_len dimension, 
v = torch.cat((past_value, new_v), dim='seq_len')
```


Of the 8 `F.linear` ops performed by each DecoderLayer was doing, we've managed to completely cache 2 of them, and almost completely cache 2 more. Overall, we chop off about 40% of the runtime. `TODO(SS): verify`.



[^2]: 



### Cut

> Note 
Most of our other models do not make inputs for the user -- that's the tokenizer's job, but as the t5 authors write:

> "A major factor that differentiates the architectures is the mask used by different attention mechanisms in the model." 
